# Contribute an MP App

The most important libraries to build an MP app are: dash, dash-mp-components, and crystal toolkit. 
Depending on how you like to design your app and what components your app need, there are two ways to write it.

1. If you want to use existing components from crystal toolkit, such as the structure viewer, band structure viewer, you can subclass `MPComponent` from `ctk`

2. If you don't need any ctk components, you just want to write plain dash layout, you can use the Dash [all-in-one component](https://dash.plotly.com/all-in-one-components) convention. 

In the future, we hope to unite these two methods to be able to use the all-in-one convention interchangeably. But for now, we need to differentiate it. 
Two examples will be given below for two scenarios.


## Use crystal toolkit

`ctk` comes with an array of components that work well with MSONable objects, such as structures, bandstructures, phonon dispersions, XRD spectrum etc. If your app benefits from any of these components, it is recommended you using the following format. 

### Example

In [24]:
import dash
from dash import html

# standard Dash imports for callbacks (interactivity)
from dash import Input, Output, no_update, State
from pymatgen.core.lattice import Lattice
from pymatgen.core.structure import Structure
from mp_api.client import MPRester
import crystal_toolkit.components as ctc
import crystal_toolkit.helpers.layouts as ctl
from crystal_toolkit.core.mpcomponent import MPComponent 

class MyComponent(MPComponent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # An interactie structure component 
        # note the usage of self.id("my_structure"), this is important to prevent id name clash 
        # an interactive ctk component needs to be instantiated in the __init__ function
        # now the data property is empty because it doesn't know what structure to display yet
        self.structure_component = ctc.StructureMoleculeComponent(id=self.id("my_structure")) 

    def layout(self):
        # A search bar for using mpid 
        search_input = ctl.Input(placeholder="material id", type="text",id=self.id("my_input"))
        button = ctl.Button("Search", id=self.id("my_button"))
        message = html.Div(id = self.id("my_message"))
        return html.Div([search_input, button, self.structure_component.layout(), message])
    
    def generate_callbacks(self, app, cache):
        super().generate_callbacks(app, cache)
        @app.callback(
            Output(self.id("my_structure"), "data"),
            Output(self.id("my_message"), "children"),
            State(self.id("my_input"), "value"), 
            # we use State here because we don't want the callback to trigger 
            # everytime a user types text in the field
            Input(self.id("my_button"), "n_clicks")
        )
        def update_structure(mpid, n_clicks):
            if not n_clicks:
                return no_update, no_update
            mpr = MPRester()
            try:
                docs = mpr.materials.summary.search(material_ids=[mpid])
            except Exception as e: 
                return no_update, "Error finding the structure"
            
            if not docs:
                return no_update, f"There is no {mpid}"
            
            structure = docs[0].structure
            
            return structure, "Found the structure"
            

if __name__ == "__main__":
    app = dash.Dash(__name__)
    layout = MyComponent().layout()
    ctc.register_crystal_toolkit(app=app, layout=layout, cache=None)
    app.run_server(debug=True, port=8050)

Retrieving SummaryDoc documents: 0it [00:00, ?it/s]

# Using dash all-in-one components

If you don't need `ctk` structure components or other components alike, you can directly use the Dash framework to build your app. Here, we strongly recommend you using the "all-in-one" components convention as it is a better choice for our infrastracture.

### Example

In [33]:

from dash import html, dcc, callback, Input, Output, State, MATCH, no_update, dash_table, Patch 
from dash.exceptions import PreventUpdate
import dash_ag_grid as dag
import pandas as pd
from mp_api.client import MPRester
import dash
import altair as alt
import dash_vega_components as dvc
import plotly.graph_objects as go
import json
import uuid

class MPAppAIO(html.Div): # we use the Dash recommended AIO as name convention
    # Here, we put all the needed components in a ids class upfront, with a dictionary 
    # as the name for each. So we can use the "pattern-matching" in the callbacks. 
    class ids:
        search_bar = lambda aio:{
            "component": "MaterialsGraphAIO",
            "aio": aio,
            "subcomponents": "searchUIAIO"
        } 
        quickFilter = lambda aio:{
            "component": "MaterialsGraphAIO",
            "aio": aio,
            "subcomponents": "quickFilter"
        }
        button = lambda aio:{
            "component": "MaterialsGraphAIO",
            "aio": aio,
            "subcomponents": "button"
        }
        datatable = lambda aio:{
            "component": "MaterialsGraphAIO",
            "aio": aio,
            "subcomponents": "datatable"
        }

    # Expose the ids from the class 
    ids = ids 

    def __init__(self, id=None, aio=None, **kwargs):
        if aio is None:
            # Otherwise use a uuid that has virtually no chance of collision.
            # Uuids are safe in dash deployments with processes
            # because this component's callbacks
            # use a stateless pattern-matching callback:
            # The actual ID does not matter as long as its unique and matches
            # the PMC `MATCH` pattern..
            aio = str(uuid.uuid4())
        self.aio = aio
        self.kwargs = kwargs

        searchbar = html.Div(
            [
            dcc.Input(id=self.ids.search_bar(self.aio), type='text', placeholder='Enter chemical system'),
            html.Button('Submit', n_clicks=0,  id=self.ids.button(self.aio))
            ]
            )
        quick_filter = dcc.Input(id=self.ids.quickFilter(self.aio), type='text', placeholder='Quick Filter')

        datatable = dag.AgGrid(id=self.ids.datatable(self.aio), dashGridOptions={'pagination':True},)

        super().__init__(children=[
            searchbar,
            quick_filter,
            datatable
        ], **kwargs)

    # callback for update the data table filter value 
    @callback(
        Output(ids.datatable(MATCH), "dashGridOptions"),
        Input(ids.quickFilter(MATCH), "value"),
        allow_duplicate=True,
    )
    def update_filter(filter_value):
        newFilter = Patch()
        newFilter['quickFilterText'] = filter_value
        return newFilter

    # callback for updating the content in a table
    @callback(
        Output(ids.datatable(MATCH), "rowData"),
        Output(ids.datatable(MATCH), "columnDefs"),
        Input(ids.button(MATCH), "n_clicks"),
        State(ids.search_bar(MATCH), "value"),
        prevent_initial_call=True,
        allow_duplicate=True,

    )
    def update_datatable(n_clicks, value):
        if not n_clicks:
            return no_update
        mpr = MPRester()
        docs = mpr.materials.summary.search(chemsys=value)

        doc_list = [_clean_dict(doc.model_dump()) for doc in docs]

        df = pd.DataFrame(doc_list)
        column_defs = [{"field": i} for i in df.columns]

        return df.to_dict("records"), column_defs

# Util function
def _clean_dict(d):
    """
    Remove fields with None values or custom object types from the dictionary.
    """
    cleaned_dict = {}
    for key, value in d.items():
        if value is not None and not isinstance(value, (Structure, dict, Composition, Lattice, list)) :
            cleaned_dict[key] = value
    return cleaned_dict


if __name__ == "__main__":
    app = dash.Dash(__name__, suppress_callback_exceptions=True, use_pages=False)
    app.layout = html.Div(MPAppAIO(aio="test"))
    app.run_server(debug=True)